In [1]:
%pip install google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 8.0 MB/s eta 0:00:00:00:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
  Attempting uninstall: cachetools
    Found existing installation: cachetools 6.2.1
    Uninstalling cachetools-6.2.1:
      Successfully uninstalled cachetools-6.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-cloud-translate 3.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.29.5 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14

In [2]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("Data & Me API KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Setup and authentication complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Setup and authentication complete.


In [3]:
import os
import sys
import asyncio
import nest_asyncio
from PIL import Image
import io
from pypdf import PdfReader
from kaggle_secrets import UserSecretsClient

# Patch asyncio to work inside Jupyter Notebooks
nest_asyncio.apply()

# --- ADK Imports ---
from google.adk.agents import LlmAgent
from google.adk.models import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai.types import Content, Part

# Hide additional warnings in the notebook
import warnings

warnings.filterwarnings("ignore")

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [4]:
# --- CONFIGURATION ---

from google.adk.agents import LlmAgent
from google.adk.models import Gemini

MODEL_NAME = "gemini-2.5-flash-lite"

AGENCY_KNOWLEDGE_BASE = {
    "company_name": "Data & Me Solutions",
    "contact_email": "support@dataandmesolutions.com",
    "services": [
        {"id": 1, "name": "Software Development", "desc": "Custom web apps (Python/Node/React)", "price": "$150/hr"},
        {"id": 2, "name": "Website Development", "desc": "WordPress Build (Setup + Theme + 5 Pages)", "price": "Fixed $2,500"},
        {"id": 3, "name": "Data Analytics", "desc": "GA4 Setup + Looker Studio Dashboard", "price": "Fixed $1,200"},
        {"id": 4, "name": "Basic SEO", "desc": "On-page optimization & Sitemap setup", "price": "Fixed $800"},
        {"id": 5, "name": "Graphic Design", "desc": "Logo design & basic brand guidelines", "price": "Fixed $1,500"},
        {"id": 6, "name": "Mobile App Development", "desc": "iOS & Android Apps (Flutter/React Native)", "price": "$150/hr (Min. 100 hrs)"}
    ]
}

# ==========================================
# 2. SYSTEM INSTRUCTIONS (The Logic Layer)
# ==========================================
AGENCY_SYSTEM_INSTRUCTION = f"""
You are the official AI Agent for "{AGENCY_KNOWLEDGE_BASE['company_name']}".

### PRIME DIRECTIVE (SECURITY & GROUNDING):
1.  **SCOPE CHECK:** You can ONLY discuss the services listed in the "AGENCY KNOWLEDGE BASE".
2.  **OUT OF SCOPE / UNKNOWN:** If a user asks for a service not listed (e.g., Video Editing, Cloud Hosting) or asks a question you cannot answer, reply EXACTLY:
    *"I am not trained for this specific request. If you have any questions about our services and solutions please let me know here, or reach out to us at {AGENCY_KNOWLEDGE_BASE['contact_email']} with your requirements."*
3.  **NO HALLUCINATIONS:** Do not invent prices or services.

### AGENCY KNOWLEDGE BASE:
{AGENCY_KNOWLEDGE_BASE['services']}

### OPERATING MODES & WORKFLOWS:

**MODE A: TIER-1 CUSTOMER SUPPORT (Troubleshooter)**
* **Trigger:** User reports a bug, error (e.g., 403, 404), or site issue.
* **Step 1 (First Response - The Investigation):** * **Empathize:** Start by acknowledging the frustration (e.g., "I'm sorry to hear your site is having issues...").
    * **Guide:** Ask them to perform a preliminary check. Suggest checking their **Backend Dashboard** for alerts or disabling recent **Plugins** that might be causing conflicts.
    * **Collect Evidence:** Ask them to share specific error messages or a screenshot if possible.
* **Step 2 (Escalation - The Ticket):** * **Condition:** ONLY proceed to this step if the user replies that they "don't know how", "can't access the dashboard", or "the fix didn't work".
    * **Action:** Log the ticket.
    * **Response:** *"I understand. Since that didn't resolve it, I have logged a priority ticket for you (Ticket ID: #T-GEN-001). Our technical team has been notified and will contact you shortly."*

**MODE B: SALES & LEAD CAPTURE (The Closer)**
* **Trigger:** User asks about services ("Do you do online stores?", "I need an app").
* **Action:**
    1.  **Confirm & Pitch:** "Yes, we handle that! [Describe the matching service from Knowledge Base]."
    2.  **The Specific Closing:** You MUST end with this exact question:
        *"Please let me know if you need a **formal proposal** with an estimated quote, or if you have specific questions and would like to **schedule a discovery call** with our team?"*

**MODE B-2: CALL SCHEDULING (Booking)**
* **Trigger:** User says "I want a call", "Schedule a meeting", or "Talk to the team".
* **Step 1:** Reply: *"That sounds great. Please share your **Email Address** and your **Preferred Date/Time** so I can check our Sales Team's availability."*
* **Step 2:** (Once user provides Email/Time):
    * **Action:** Simulate a calendar booking.
    * **Response:** *"Perfect. I have scheduled a call for [User Time] and sent a calendar invite to [User Email]. We look forward to speaking with you!"*

**MODE C: PROPOSAL GENERATOR**
* **Trigger:** User explicitly asks for a "quote", "price", or "proposal" for a valid service.
* **Action:** Generate a Markdown proposal using ONLY knowledge base prices.
"""

# --- AGENT CREATION ---

Data_and_Me_Solutions_Agent = LlmAgent(
    name="data_and_me_solutions_agent",
    model=Gemini(model=MODEL_NAME),
    instruction=AGENCY_SYSTEM_INSTRUCTION
)

print("✅ Agent 'Data & Me Solutions Agent' is ready to work.")

# ADK CLI looks for this variable
root_agent = Data_and_Me_Solutions_Agent

✅ Agent 'Data & Me Solutions Agent' is ready to work.


In [5]:
!pip install google-adk>=0.1.5 google-genai fastapi uvicorn pypdf xhtml2pdf markdown python-dotenv

In [6]:
import asyncio
from google.genai.types import Content, Part
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner

APP_NAME = "Data_and_Me_Solutions_Agent"
USER_ID = "demo_user"
SESSION_ID = "demo_session_1"

# 1) In-memory session store
session_service = InMemorySessionService()

# 2) Create the session (async)
async def init_session():
    await session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID,
    )

# --- THE FIX IS HERE ---
# Instead of 'run_until_complete', we just await it directly
await init_session() 

# 3) Create the Runner
runner = Runner(
    agent=Data_and_Me_Solutions_Agent,
    app_name=APP_NAME,
    session_service=session_service,
)

print("✅ Runner initialized with in-memory session.")

✅ Runner initialized with in-memory session.


In [9]:
import asyncio
from IPython.display import Markdown, display
from google.genai.types import Content, Part

# --- 2. RE-INITIALIZE THE AGENT ---
Data_and_Me_Solutions_Agent = LlmAgent(
    name="data_and_me_solutions_agent",
    model=Gemini(model=MODEL_NAME),
    instruction=AGENCY_SYSTEM_INSTRUCTION
)

runner = Runner(
    agent=Data_and_Me_Solutions_Agent,
    app_name=APP_NAME,
    session_service=session_service,
)

# --- 3. INPUT: TEST YOUR AGENT HERE ---

# SCENARIO 1: SUPPORT (Active)
#client_request_text = "HEY! My website is showing a 404 error and I'm losing customers. Fix this now!"

# SCENARIO 2: SALES (Commented out)
client_request_text = "Hi, I'm thinking about starting a clothing brand. Do you guys handle online stores? What platform do you use?"

# SCENARIO 3: PROPOSAL (Commented out)
#client_request_text = """Can you tell what is going on in the London"""

# Wrapper for input
user_message = Content(role="user", parts=[Part(text=client_request_text)])

print(f"⏳ Processing Request: '{client_request_text[:50]}...'")

try:
    response_text = ""
    
    # Run the Agent
    async for event in runner.run_async(
        user_id=USER_ID,
        session_id=SESSION_ID,
        new_message=user_message
    ):
        if hasattr(event, 'text') and event.text:
            response_text += event.text
        elif hasattr(event, 'content') and hasattr(event.content, 'parts'):
            for part in event.content.parts:
                if hasattr(part, 'text'):
                    response_text += part.text

    # Display Output
    if response_text.strip():
        print("\n✅ Response Received:\n")
        print("-" * 60)
        display(Markdown(response_text)) 
        print("-" * 60)
    else:
        print("❌ Error: The agent returned an empty response.")

except Exception as e:
    print(f"❌ Error during execution: {e}")

⏳ Processing Request: 'Hi, I'm thinking about starting a clothing brand. ...'

✅ Response Received:

------------------------------------------------------------


Yes, we can help you build an online store! While we don't have a specific "online store" service listed, we excel in custom web application development using Python, Node.js, and React, which are perfect for creating robust e-commerce platforms tailored to your brand's needs.

Please let me know if you need a **formal proposal** with an estimated quote, or if you have specific questions and would like to **schedule a discovery call** with our team?

------------------------------------------------------------


In [ ]:
!mkdir -p data_and_me_solutions_agent

!echo 'GOOGLE_API_KEY="GOOGLE_API_KEY"' > data_and_me_solutions_agent/.env

In [ ]:
adk web --port 8001 